In [1]:
#numerical / data packages
import numpy as np
import pandas as pd
import pytraj as pt
import scipy as sp

#utilities
import os
import sys
import gc
import copy
import tqdm
import time
#from tqdm import notebook

#interactive widgets
#import ipywidgets as widgets
#from ipywidgets import interact, interact_manual

#matplotlib plotting
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns

#plotly plotting
import plotly as ply
from plotly import graph_objs as go
ply.io.renderers.default="notebook"

#%cd /data/lyna
#import correlation_data_utilities as corr_utils

#tqbar=tqdm.notebook.tqdm
tqbar=tqdm.tqdm_notebook

In [5]:
# Example adjacency network on Botello-Smith and Luo, JCTC2019, Figure 2
Adj=sp.sparse.coo_matrix(
    (
        np.array(
            [.25,.33,.5,1,.25,.25,.33,.33,.5,.5,1,
             1,.25,.25,.33,.33,.5,.5,1,1,.25,.33,.5,1]),
        ([0,0,0,0,1,1,2,2,3,3,4,4,5,5,6,6,7,7,8,8,9,9,9,9],
         [1,2,3,4,0,5,0,6,0,7,0,8,1,9,2,9,3,9,4,9,5,6,7,8]
        )
    )
)

AdjDense=Adj.todense()
print(AdjDense)

[[0.   0.25 0.33 ... 0.   0.   0.  ]
 [0.25 0.   0.   ... 0.   0.   0.  ]
 [0.33 0.   0.   ... 0.   0.   0.  ]
 ...
 [0.   0.   0.   ... 0.   0.   0.5 ]
 [0.   0.   0.   ... 0.   0.   1.  ]
 [0.   0.   0.   ... 0.5  1.   0.  ]]


In [6]:
# Convert adjacency matrix to Laplacian matrix
LapDense=-copy.deepcopy(AdjDense)
for ii, irow in enumerate(LapDense):
    LapDense[ii,ii]=-np.sum(irow)
print(LapDense)

[[ 2.08 -0.25 -0.33 ... -0.   -0.   -0.  ]
 [-0.25  0.5  -0.   ... -0.   -0.   -0.  ]
 [-0.33 -0.    0.66 ... -0.   -0.   -0.  ]
 ...
 [-0.   -0.   -0.   ...  1.   -0.   -0.5 ]
 [-0.   -0.   -0.   ... -0.    2.   -1.  ]
 [-0.   -0.   -0.   ... -0.5  -1.    2.08]]


In [7]:
# Compute Moore-Penrose pseudoinverse of Laplacian matrix (results will depend on numpy version)
LapInv=np.linalg.pinv(LapDense)
LapInv

matrix([[ 0.56118298, -0.07920163,  0.01776807, ..., -0.11958625,
         -0.01958625, -0.15997086],
        [-0.07920163,  2.10733683, -0.46236014, ..., -0.43945804,
         -0.33945804, -0.31958625],
        [ 0.01776807, -0.46236014,  1.65481158, ..., -0.34248834,
         -0.24248834, -0.22261655],
        ...,
        [-0.11958625, -0.43945804, -0.34248834, ...,  1.1740035 ,
         -0.05932984,  0.12079837],
        [-0.01958625, -0.33945804, -0.24248834, ..., -0.05932984,
          0.70733683,  0.22079837],
        [-0.15997086, -0.31958625, -0.22261655, ...,  0.12079837,
          0.22079837,  0.56118298]])

In [8]:
# Compute betweenness for edge 4-8 when source and sink nodes are 1 and 10 using eqs in Fig2
# results should agree with Kirchhoff's law in Fig1
v_1_10_4=LapInv[3,0]-LapInv[3,9]
v_1_10_8=LapInv[7,0]-LapInv[7,9]
b_4_8=AdjDense[3,7]*(v_1_10_4 - v_1_10_8)

print(v_1_10_4,v_1_10_8,b_4_8)

0.24038461538461636 -0.240384615384615 0.2403846153846157


In [40]:
# (v_1_10_4 - v_1_10_8) can also be obtained by 
# multiple LapInv by column vector ColVec and row vector RowVec 

ColVec=np.matrix(np.atleast_2d(np.array([0,0,0,1,0,0,0,-1,0,0])))
RowVec=np.matrix(np.atleast_2d(np.array([1,0,0,0,0,0,0,0,0,-1])).T)
Btw_4_8=AdjDense[3,7]*ColVec*LapInv*RowVec
print(ColVec, RowVec, Btw_4_8)

[[ 0  0  0  1  0  0  0 -1  0  0]] [[ 1]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [-1]] [[0.24038462]]


In [58]:
# For large matrix, conjugated gradient can be used instead of pseudo-inverse to compute betweeness
sources=[0] # can have multiple source and sink nodes
sinks=[9]
loadMat=np.zeros((len(LapInv),len(sources)*len(sinks)))

ii=0
for isource, source in enumerate(sources):
    for isink, sink in enumerate(sinks):
        print(isource, "source node:", source)
        print(isink, "sink node", sink)
        loadMat[source, ii]=1
        loadMat[sink,ii]=-1
        ii+=1
with np.printoptions(threshold=np.inf):
        print(loadMat)

0 source node: 0
0 sink node 9
[[ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]]


In [60]:
LapSparse=sp.sparse.coo_matrix(LapDense)
print(LapDense)
print(LapSparse)

[[ 2.08 -0.25 -0.33 ... -0.   -0.   -0.  ]
 [-0.25  0.5  -0.   ... -0.   -0.   -0.  ]
 [-0.33 -0.    0.66 ... -0.   -0.   -0.  ]
 ...
 [-0.   -0.   -0.   ...  1.   -0.   -0.5 ]
 [-0.   -0.   -0.   ... -0.    2.   -1.  ]
 [-0.   -0.   -0.   ... -0.5  -1.    2.08]]
  (0, 0)	2.08
  (0, 1)	-0.25
  (0, 2)	-0.33
  (0, 3)	-0.5
  (0, 4)	-1.0
  (1, 0)	-0.25
  (1, 1)	0.5
  (1, 5)	-0.25
  (2, 0)	-0.33
  (2, 2)	0.66
  (2, 6)	-0.33
  (3, 0)	-0.5
  (3, 3)	1.0
  (3, 7)	-0.5
  (4, 0)	-1.0
  (4, 4)	2.0
  (4, 8)	-1.0
  (5, 1)	-0.25
  (5, 5)	0.5
  (5, 9)	-0.25
  (6, 2)	-0.33
  (6, 6)	0.66
  (6, 9)	-0.33
  (7, 3)	-0.5
  (7, 7)	1.0
  (7, 9)	-0.5
  (8, 4)	-1.0
  (8, 8)	2.0
  (8, 9)	-1.0
  (9, 5)	-0.25
  (9, 6)	-0.33
  (9, 7)	-0.5
  (9, 8)	-1.0
  (9, 9)	2.08


In [65]:
# conjugated gradient solver takes in a sparse matrix and a vector as input

potVec=np.zeros(Adj.shape[0])
print("initial potVec", potVec)
for loadVec in loadMat.T:
    print("loadVec",loadVec) # loadMat = RowVec
    
    # equation below is equal to potVec=np.sum(LapInv*loadMat)
    potVec=potVec+sp.sparse.linalg.cg(LapSparse,loadVec)[0]
    
    potVec=potVec/len(loadMat.T)
    print("potVec", potVec, len(loadMat.T))

btw_4_8=np.abs(Adj.tocsr()[3,7]*(potVec[7]-potVec[3]))
print('btw_4_8',btw_4_8)

initial potVec [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
loadVec [ 1.  0.  0.  0.  0.  0.  0.  0.  0. -1.]
potVec [ 0.72115385  0.24038462  0.24038462  0.24038462  0.24038462 -0.24038462
 -0.24038462 -0.24038462 -0.24038462 -0.72115385] 1
btw_4_8 0.24038461538461536


# Test case ends here

# Below is using test case to check the corr_utils functions

In [15]:
sourceInds=[3]
targetInds=[7]
print(networkAdj[sourceInds,targetInds])

btw1=np.array(corr_utils.getBtwMat(
    mat=copy.deepcopy(networkAdj),
    sources=sourceInds,targets=targetInds,
    pbarFun=tqbar, verbose=True, verboseLevel=15,useLegacyAlgorithm=True))

with np.printoptions(threshold=np.inf):
    print("~btw1~")
    print(btw1)
    


print('~~~~~~~~~~~~~~~~~~~~~~~~~~~')
print('new method')
netLap=corr_utils.matLap(corr_utils.matAdj(copy.deepcopy(networkAdj)))
with np.printoptions(threshold=np.inf):
    print("~netLap~")
    print(netLap)

lapInv=np.linalg.pinv(netLap, hermitian=True)
with np.printoptions(threshold=np.inf):
    print("~lapInv~")
    print(lapInv)


[0.5]
computing matrix Laplacian
[[ 2.08 -0.25 -0.33 ... -0.   -0.   -0.  ]
 [-0.25  0.5  -0.   ... -0.   -0.   -0.  ]
 [-0.33 -0.    0.66 ... -0.   -0.   -0.  ]
 ...
 [-0.   -0.   -0.   ...  1.   -0.   -0.5 ]
 [-0.   -0.   -0.   ... -0.    2.   -1.  ]
 [-0.   -0.   -0.   ... -0.5  -1.    2.08]]
extracting weighted adjacency matrix
[[0.   0.25 0.33 ... 0.   0.   0.  ]
 [0.25 0.   0.   ... 0.   0.   0.  ]
 [0.33 0.   0.   ... 0.   0.   0.  ]
 ...
 [0.   0.   0.   ... 0.   0.   0.5 ]
 [0.   0.   0.   ... 0.   0.   1.  ]
 [0.   0.   0.   ... 0.5  1.   0.  ]]
computing moore-penrose inverse of matrix Laplacian
[[ 0.56118298 -0.07920163  0.01776807 ... -0.11958625 -0.01958625
  -0.15997086]
 [-0.07920163  2.10733683 -0.46236014 ... -0.43945804 -0.33945804
  -0.31958625]
 [ 0.01776807 -0.46236014  1.65481158 ... -0.34248834 -0.24248834
  -0.22261655]
 ...
 [-0.11958625 -0.43945804 -0.34248834 ...  1.1740035  -0.05932984
   0.12079837]
 [-0.01958625 -0.33945804 -0.24248834 ... -0.05932984  0.